In [3]:
import os
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
blob_container_name = os.environ["BLOB_CONTAINER_NAME"]

In [5]:
from azure.storage.blob import BlobServiceClient  
import os

# Connect to Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client(blob_container_name)
if not container_client.exists():
    container_client.create_container()

deprecated_blobs = [blob for blob in container_client.list_blob_names()]

print(f"Setup sample data in {blob_container_name}...")

documents_directory = os.path.join("data")
for file in os.listdir(documents_directory):
    with open(os.path.join(documents_directory, file), "rb") as data:
        name = os.path.basename(file)
        if not container_client.get_blob_client(name).exists():
            print(f'Uploading: {name}')
            container_client.upload_blob(name=name, data=data)

        if name in deprecated_blobs:
                deprecated_blobs.remove(name)

if len(deprecated_blobs) > 0:
    for _blob_name in deprecated_blobs:
        print(f"Deleting (not found locally): {_blob_name}")
        container_client.delete_blob(_blob_name)

Setup sample data in aihackathon...
Uploading: form-A-page0.png
